In [ ]:
import os
import seisbench.models as sbm
from obspy import UTCDateTime as utc
import pickle
from datetime import datetime as dt
import pandas as pd
import configparser as cnfp
import SeisRoutine.helper as srh
from pathlib import Path

In [ ]:
config_path = 'config.ini'
config = cnfp.ConfigParser(interpolation=cnfp.ExtendedInterpolation())
config.read(config_path)

time_name = utc().strftime('%Y%m%dT%H%M%S')
config.set('General', 'last_run_time', time_name)
with open(config_path, 'w') as configfile:
    config.write(configfile)

In [ ]:
config.read(config_path)
#
wave_extension = config.get('Params', 'data_extension')
processor = config.get('Params', 'processor')
DPDN = eval(config.get('Params', 'DPDN'))

wave_dir = config.get('Path', 'wave')
pick_dir = config.get('Path', 'picks')
pick_dir

In [ ]:
low_corner_filter = [None]
high_corner_filter = [None]

In [ ]:
root_output = f'{pick_dir}/sbpicks'
os.makedirs(pick_dir, exist_ok=True)

Loading Deep-Learning Models

In [ ]:
if DPDN:
    model_dnoise = sbm.DeepDenoiser.from_pretrained('original')
    if processor == 'GPU':
        model_dnoise.cuda()

In [ ]:
# PhaseNet, EQTransformer, BasicPhaseAE, GPD
df_dl_models = pd.DataFrame(
    columns=['model_name', 'data_name', 'model'],
    data=[
        ['PhaseNet', 'original', None],
        ['EQTransformer', 'original', None]
    ]
)

for indx, row in df_dl_models.iterrows():
    row.model = eval(f"sbm.{row.model_name}.from_pretrained(\'{row.data_name}\')")
    if processor == 'GPU':
        row.model.cuda()

# print(df_dl_models)

Data Processing

In [ ]:
meta_header = ['date_data', 'wavepath', 'net_sta', 'runtime',
               'freqmin', 'freqmax', 'DPDN',
               'model.name', 'model.model.name', 'model.data_name', 'model.detection_threshold',
               'num_of_picks']
with open(f'{root_output}/metadata-run.csv', 'a') as metaout:
    metaout.write(','.join(meta_header)+'\n')

In [ ]:
for freqmin, freqmax in zip(low_corner_filter, high_corner_filter):
    for root, dirs, files in os.walk(wave_dir):
        # filter files with specific extensions and extract target dates
        files = [f for f in files if f.endswith(wave_extension)]
        dates = {f.split('_')[0] for f in files}
        dates = sorted(list(dates))
        # Reading data in each date
        for date_data in dates:
            wavepath = f'{root}/{date_data}*.{wave_extension}'
            # print(wavepath)
            st = srh.read_gcf(path=wavepath)
            if len(st)==0:
                continue
            # Preprocessing
            st.merge(-1)
            st.detrend('constant')
            st.merge(fill_value=0)
            #
            tr = st[0]
            net_sta = f"{tr.stats.network}.{tr.stats.station}"
            # Filtering
            if freqmin or freqmax:
                print(f'Filtering: {freqmin=} {freqmax=}')
                st.filter('bandpass',
                          freqmin=freqmin, freqmax=freqmax,
                          zerophase=True)
            if DPDN:
                st = model_dnoise.annotate(st)
            # Applying Deep Learning models
            for indx, model in df_dl_models.iterrows():
                runtime_start = dt.now()
                classified = model.model.classify(st)
                runtime_end = dt.now()
                runtime = (runtime_end-runtime_start).total_seconds()
                ### write results in an output file
                num_of_picks = len(classified.picks)
                runtime_name = runtime_start.strftime('%Y%m%dT%H%M%S')
                fname = f'ST-{net_sta}_DD-{date_data}_M-{model.model.name}_T-{runtime_name}.pickle'
                outpath = f'{root_output}/{fname}'
                with open(outpath, 'wb') as outfile:
                    pickle.dump(classified, outfile)
                ###
                with open(f'{root_output}/metadata-run.csv', 'a') as metaout:
                    meta = []
                    for _ in meta_header:
                        try:
                            meta.append(str(eval(_)))
                        except:
                            meta.append('None')
                    metaout.write(','.join(meta)+'\n')